In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
#from loguru import logger
from typing import Union, Optional
from fastapi.middleware.cors import CORSMiddleware

from oct2py import octave
import os
import fnmatch
import json

octave.run('BattMo/startupBattMo.m')

main_app = FastAPI()
app = FastAPI(title="BattMo API", description="runs BattMo Simulations as a service", version="0.1.0")

origins = [
    "*",
    "https://api.battmo.open-semantic-lab.org",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class TestRequest(BaseModel):
    text: str

class ActiveMaterialClass(BaseModel):
    thickness: float
    
class SeparatorClass(BaseModel):
    thickness: float
    
class NegativeElectrodeClass(BaseModel):
    ActiveMaterial: ActiveMaterialClass
    
class PositiveElectrodeClass(BaseModel):
    ActiveMaterial: ActiveMaterialClass
    
class ElectrolyteClass(BaseModel):
    Separator: SeparatorClass

class Geometry1D(BaseModel):
    format: Optional[str] = "1D"
    faceArea: Optional[float] = 1e-4
    NegativeElectrode: Optional[NegativeElectrodeClass] = NegativeElectrodeClass(ActiveMaterial = ActiveMaterialClass(thickness = 64e-6))
    PositiveElectrode: Optional[PositiveElectrodeClass] = PositiveElectrodeClass(ActiveMaterial = ActiveMaterialClass(thickness = 57e-6))
    Electrolyte: Optional[ElectrolyteClass] = ElectrolyteClass(Separator = SeparatorClass(thickness = 15e-6))
    
class PerformanceSpec(BaseModel):
    E: Optional[float]
    energyDensity: Optional[float]
    energy: Optional[float]
    
class PerformanceSpecRequest(BaseModel):
    geometry: Optional[Geometry1D] = Geometry1D()
    
class PerformanceSpecResponse(BaseModel):
    status: Optional[str] = "ok"
    result: PerformanceSpec

@app.post("/run/performance-spec", response_model=PerformanceSpecResponse)
def performance_spec(request: PerformanceSpecRequest):
    #ToDo
    
    print(request.geometry.json())
    
     # count the number of files in the input directory for file naming (sequential numbering with newest identified as New)
    input_dir_path = r'BattMo/Examples/experiment/optimization_test/input_json/'
    input_count = len(fnmatch.filter(os.listdir(input_dir_path), '*.*'))
    if input_count > 1:
        old_name = r'BattMo/Examples/experiment/optimization_test/input_json/geometryParametersNew.json'
        new_name = r'BattMo/Examples/experiment/optimization_test/input_json/geometryParameters'+str(input_count-1)+'.json'
        os.rename(old_name, new_name)
        battmo_input = 'BattMo/Examples/experiment/optimization_test/input_json/geometryParametersNew.json'
    else:
        battmo_input = 'BattMo/Examples/experiment/optimization_test/input_json/geometryParametersNew.json'
            
    f = open(battmo_input, "w")
    f.write(request.geometry.json())
    f.close()
    
    # identify the input json file containing the geometry parameters
    # ToDo why does BattMo require you to restate it like this? Why not take the direct path?
    battmo_input = 'Examples/experiment/optimization_test/input_json/geometryParametersNew.json'
    
    # count the number of files in the output directory for file naming (sequential numbering with last identified as Last)
    output_dir_path = r'BattMo/Examples/experiment/optimization_test/output_json/'
    output_count = len(fnmatch.filter(os.listdir(output_dir_path), '*.*'))
    if output_count > 1:
        old_name = r'BattMo/Examples/experiment/optimization_test/output_json/performanceSpecLast.json'
        new_name = r'BattMo/Examples/experiment/optimization_test/output_json/performanceSpec'+str(output_count-1)+'.json'
        os.rename(old_name, new_name)
        battmo_output = 'BattMo/Examples/experiment/optimization_test/output_json/performanceSpecLast.json'
    else:
        battmo_output = 'BattMo/Examples/experiment/optimization_test/output_json/performanceSpecLast.json'
        
    # run the simulation
    E, energyDensity, energy = octave.runJsonFunction({'ParameterData/BatteryCellParameters/LithiumIonBatteryCell/lithium_ion_battery_nmc_graphite.json', battmo_input}, battmo_output, nout=3)
    response = PerformanceSpecResponse(result=PerformanceSpec(E = E[:,0][-1], energyDensity = energyDensity[:,0][-1], energy = energy[:,0][-1]))
    return response

main_app.mount("/api", app)

import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(main_app, port=8200, host="0.0.0.0")

#for testing open https://api.battmo.open-semantic-lab.org/api/docs#/default/performance_spec_run_performance_spec_post

Welcome to the MATLAB Reservoir Simulation Toolbox (MRST)!
You are using the release version 2021b. To download other versions of MRST
and view examples and relevant publications, please visit www.mrst.no

Useful commands for getting started:
 - List all introductory examples:   mrstExamples()
 - List all modules:                 mrstPath('list')
 - Load modules using GUI:           mrstModule('gui')
 - Explore all available data sets:  mrstDatasetGUI()
 - List examples of a module:        mrstExamples('ad-blackoil')
 - Explore modules and publications: mrstExploreModules()
 - Show all examples in all modules: mrstExamples('all')
 - Show settings for MRST:           mrstSettings()
 - Display this message:             mrstStartupMessage()

For assistance and discussions about MRST, please visit our mailing list at
	www.sintef.no/projectweb/mrst/forum/ (sintef-mrst@googlegroups.com)
For some common queries, see our FAQ: www.sintef.no/projectweb/mrst/faq/


INFO:     Started server process [1932473]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8200 (Press CTRL+C to quit)


{"format": "1D", "faceArea": 0.0001, "NegativeElectrode": {"ActiveMaterial": {"thickness": 6.4e-05}}, "PositiveElectrode": {"ActiveMaterial": {"thickness": 5.7e-05}}, "Electrolyte": {"Separator": {"thickness": 1.5e-05}}}
ans = 0
Solving timestep 01/50:                      -> 100 Seconds, 799 Milliseconds
Solving timestep 02/50: 100 Seconds, 799 Milliseconds -> 201 Seconds, 599 Milliseconds
Solving timestep 03/50: 201 Seconds, 599 Milliseconds -> 302 Seconds, 399 Milliseconds
Solving timestep 04/50: 302 Seconds, 399 Milliseconds -> 403 Seconds, 199 Milliseconds
Solving timestep 05/50: 403 Seconds, 199 Milliseconds -> 504 Seconds
Solving timestep 06/50: 504 Seconds          -> 604 Seconds, 799 Milliseconds
Solving timestep 07/50: 604 Seconds, 799 Milliseconds -> 705 Seconds, 599 Milliseconds
Solving timestep 08/50: 705 Seconds, 599 Milliseconds -> 806 Seconds, 399 Milliseconds
Solving timestep 09/50: 806 Seconds, 399 Milliseconds -> 907 Seconds, 199 Milliseconds
Solving timestep 10/50: 